Black Box

In [1]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [2]:
df_train = pd.read_csv('C:\\Users\\Nishaant Harrish\\Downloads\\train.csv')
df_test = pd.read_csv('C:\\Users\\Nishaant Harrish\\Downloads\\test.csv')

In [3]:
df_train.head() 

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Splitting into training and validation dataset

In [4]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = np.array(X_train).reshape(33600, 784) #(33600, 784)
X_cv = np.array(X_cv).reshape(8400, 784) #(8400, 784)

X_test = np.array(X_test).reshape(28000, 784)

## Data cleaning, normalization and selection

In [6]:
print((min(X_train[1]), max(X_train[1])))

(0, 255)


In [7]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [8]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


## Model Fitting

In [9]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [10]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [11]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 200)               20200     
                                                                 
 Output_Layer (Dense)        (None, 10)                2010      
                                                                 
Total params: 297,910
Trainable params: 297,910
Non-trainable

In [12]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

C:\Users\Nishaant Harrish\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [14]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 6s - loss: 1.9064 - accuracy: 0.4381 - val_loss: 1.1532 - val_accuracy: 0.7182 - 6s/epoch - 17ms/step
Epoch 2/20
336/336 - 3s - loss: 0.7262 - accuracy: 0.8075 - val_loss: 0.5107 - val_accuracy: 0.8585 - 3s/epoch - 10ms/step
Epoch 3/20
336/336 - 3s - loss: 0.4392 - accuracy: 0.8751 - val_loss: 0.3846 - val_accuracy: 0.8921 - 3s/epoch - 10ms/step
Epoch 4/20
336/336 - 3s - loss: 0.3543 - accuracy: 0.8974 - val_loss: 0.3364 - val_accuracy: 0.9037 - 3s/epoch - 10ms/step
Epoch 5/20
336/336 - 3s - loss: 0.3100 - accuracy: 0.9094 - val_loss: 0.3010 - val_accuracy: 0.9148 - 3s/epoch - 10ms/step
Epoch 6/20
336/336 - 3s - loss: 0.2808 - accuracy: 0.9188 - val_loss: 0.2818 - val_accuracy: 0.9198 - 3s/epoch - 9ms/step
Epoch 7/20
336/336 - 3s - loss: 0.2575 - accuracy: 0.9244 - val_loss: 0.2556 - val_accuracy: 0.9256 - 3s/epoch - 9ms/step
Epoch 8/20
336/336 - 3s - loss: 0.2380 - accuracy: 0.9302 - val_loss: 0.2439 - val_accuracy: 0.9276 - 3s/epoch - 9ms/step
Epoch 9/20
336/336 

In [15]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

C:\Users\Nishaant Harrish\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [16]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 5s - loss: 0.3357 - accuracy: 0.8993 - val_loss: 0.1457 - val_accuracy: 0.9568 - 5s/epoch - 16ms/step
Epoch 2/20
336/336 - 3s - loss: 0.1218 - accuracy: 0.9623 - val_loss: 0.1442 - val_accuracy: 0.9581 - 3s/epoch - 10ms/step
Epoch 3/20
336/336 - 3s - loss: 0.0834 - accuracy: 0.9739 - val_loss: 0.1033 - val_accuracy: 0.9695 - 3s/epoch - 10ms/step
Epoch 4/20
336/336 - 3s - loss: 0.0618 - accuracy: 0.9793 - val_loss: 0.1002 - val_accuracy: 0.9726 - 3s/epoch - 10ms/step
Epoch 5/20
336/336 - 3s - loss: 0.0433 - accuracy: 0.9865 - val_loss: 0.1073 - val_accuracy: 0.9704 - 3s/epoch - 9ms/step
Epoch 6/20
336/336 - 3s - loss: 0.0385 - accuracy: 0.9872 - val_loss: 0.1014 - val_accuracy: 0.9704 - 3s/epoch - 10ms/step
Epoch 7/20
336/336 - 3s - loss: 0.0252 - accuracy: 0.9916 - val_loss: 0.0960 - val_accuracy: 0.9755 - 3s/epoch - 10ms/step
Epoch 8/20
336/336 - 3s - loss: 0.0296 - accuracy: 0.9904 - val_loss: 0.1021 - val_accuracy: 0.9738 - 3s/epoch - 10ms/step
Epoch 9/20
336/33

In [17]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 6s - loss: 0.3344 - accuracy: 0.9021 - val_loss: 0.1560 - val_accuracy: 0.9530 - 6s/epoch - 17ms/step
Epoch 2/20
336/336 - 3s - loss: 0.1229 - accuracy: 0.9616 - val_loss: 0.1142 - val_accuracy: 0.9638 - 3s/epoch - 10ms/step
Epoch 3/20
336/336 - 3s - loss: 0.0808 - accuracy: 0.9741 - val_loss: 0.0959 - val_accuracy: 0.9705 - 3s/epoch - 10ms/step
Epoch 4/20
336/336 - 3s - loss: 0.0577 - accuracy: 0.9821 - val_loss: 0.1372 - val_accuracy: 0.9614 - 3s/epoch - 10ms/step
Epoch 5/20
336/336 - 3s - loss: 0.0448 - accuracy: 0.9852 - val_loss: 0.0848 - val_accuracy: 0.9748 - 3s/epoch - 10ms/step
Epoch 6/20
336/336 - 3s - loss: 0.0366 - accuracy: 0.9876 - val_loss: 0.1108 - val_accuracy: 0.9702 - 3s/epoch - 10ms/step
Epoch 7/20
336/336 - 3s - loss: 0.0279 - accuracy: 0.9911 - val_loss: 0.1081 - val_accuracy: 0.9738 - 3s/epoch - 9ms/step
Epoch 8/20
336/336 - 3s - loss: 0.0278 - accuracy: 0.9910 - val_loss: 0.1143 - val_accuracy: 0.9714 - 3s/epoch - 8ms/step
Epoch 9/20
336/336

In [19]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 6s 12ms/step - loss: 0.3508 - accuracy: 0.8966 - val_loss: 0.1632 - val_accuracy: 0.9485
Epoch 2/20
336/336 [==============================] - 4s 11ms/step - loss: 0.1247 - accuracy: 0.9629 - val_loss: 0.1297 - val_accuracy: 0.9589
Epoch 3/20
 27/336 [=>............................] - ETA: 2s - loss: 0.0946 - accuracy: 0.9730

In [ ]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [ ]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

In [ ]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

In [ ]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [ ]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

In [ ]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

In [ ]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()